<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/main/af/examples/partial_hallucination_rewire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AfDesign - rewire (partial hallucination)

In [ ]:
#@title install
%%bash
if [ ! -d params ]; then
  pip -q install git+https://github.com/sokrypton/ColabDesign.git
  mkdir params
  curl -fsSL https://storage.googleapis.com/alphafold/alphafold_params_2022-03-02.tar | tar x -C params
fi

In [ ]:
#@title import libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os, re
from colabdesign.af import mk_afdesign_model, clear_mem
from IPython.display import HTML
from google.colab import files
import numpy as np

#########################
def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  else:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"

In [ ]:
# setup the model
clear_mem()
model = mk_afdesign_model(protocol="partial",
                          use_templates=False) # set True to constrain positions using template input
                        
# define positions we want to constrain (input PDB numbering)
old_pos = "4-9,13-26,30-33,37-50,54-59,63-76,80-84,112-125"

model.prep_inputs(get_pdb("2MR5"), chain="A",
                  pos=old_pos,               # define positions to contrain
                  length=100,                # define if the desired length is different from input PDB
                  fix_seq=False)             # set True to constrain the sequence

# set positions (if different from PDB)
# reorder the segments, 
model.rewire(order = [1,2,3,0,7,4,5,6], # set order of segments
             loops =  [3,3,3,4,3,3,4],  # change loop length inbetween segments
             offset=0)                  # essentially loop length at the N term

print(model.opt["pos"])

In [ ]:
# restart optimization
model.restart()
# model.set_seq(mode="wildtype") # initialize with wildtype sequence

# balance weights [dgram_cce = restraint weight], [con = hallucination weight]
model.set_weights(dgram_cce=1, con=0)
model.design_3stage(300,100,10)

In [ ]:
model.save_pdb(f"{model.protocol}.pdb")
model.plot_pdb()

In [ ]:
HTML(model.animate())